In [1]:
from langchain.chat_models import init_chat_model 
from langgraph.func import entrypoint 
from langgraph.store.memory import InMemoryStore 

from langmem import ReflectionExecutor, create_memory_store_manager 


In [2]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(
    api_key="sk-081025394d9e4f35adbb67ea07c3dae1", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="text-embedding-v4",
    check_embedding_ctx_length = False,
    dimensions=1536 
)
store = InMemoryStore(
    index={ # 存储提取的记忆 
        "dims": 1536,
        "embed": embedding,
    }
) 

In [3]:
from langchain_openai import ChatOpenAI
model_qwen=ChatOpenAI(
    api_key="sk-081025394d9e4f35adbb67ea07c3dae1", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen3-30b-a3b-instruct-2507",
)

In [17]:
# 创建记忆管理器 Runnable 来从对话中提取记忆
memory_manager = create_memory_store_manager( 
    model_qwen, 
    # 将记忆存储在 "memories" 命名空间（即目录）中
    namespace=("memories",),  
    instructions="用中文存储记忆。"
) 


In [ ]:
@entrypoint(store=store)  # 创建一个 LangGraph 工作流
async def chat(message: str): 
    response = model_qwen.invoke(message) 

    # memory_manager 从对话历史中提取记忆
    # 我们将以 OpenAI 的消息格式提供它
    to_process = {"messages": [{"role": "user", "content": message}] + [response]} 
    await memory_manager.ainvoke(to_process)  
    return response.content 

# 正常运行对话
response = await chat.ainvoke( 
    "记住我是张熙浚", 
) 
print(response) 


好的，张熙浚，我已经记住了你的名字。有什么我可以帮你的吗？


In [21]:
print(store.search(("memories",)))

[Item(namespace=['memories'], key='fe8fe312-182d-4170-869e-1cc41b44c729', value={'kind': 'Memory', 'content': {'content': "User's name is Zhang Xijun. This is a direct statement from the user in the current session, confirmed by the agent's acknowledgment. The information is explicit and unambiguous, with p(1.0). The agent has successfully stored this personal identifier for future personalized interactions."}}, created_at='2025-08-14T08:54:05.841039+00:00', updated_at='2025-08-14T08:54:05.841043+00:00', score=None), Item(namespace=['memories'], key='f19c540d-69cc-46e8-870a-91a3231e01a4', value={'kind': 'Memory', 'content': {'content': "User expressed a positive sentiment toward dogs, stating '我喜欢狗' (I like dogs). This is a direct expression of preference. The agent responded with enthusiasm and attempted to deepen the conversation by asking about pet ownership and specific dog breeds. The user's preference for dogs is explicit and unambiguous, with p(1.0). The agent has successfully r

In [22]:
len(store.search(("memories",)))

7